# Solving SAT With Grover
In this notebook, we aim to solve satisfyability formulae using grover search.
For this, we first construct marking oracles for the basic gates of AND, OR and NOT.
Then, we transform them into the required phase-oracle-form.

The main resources used are this [Microsoft Kata](https://github.com/microsoft/QuantumKatas/blob/ec925ccfcb599a6bb29c8a39035d0b149f447f9a/SolveSATWithGrover/Workbook_SolveSATWithGrover.ipynb) and this [Qiskit Textbook Chapter](https://qiskit.org/textbook/ch-applications/satisfiability-grover.html).

## Marking Oracles
### And

In [ ]:
from qiskit.quantum_info.operators import Operator
from qiskit import QuantumCircuit

## Define Basic Matrices
I = Operator([[1,0],[0,1]])
X = Operator([[0,1],[1,0]])

def sized_identity(nqubits):
    """Creates an identity operator of given size"""
    unitary = I
    for i in range(nqubits-1):
        unitary = unitary.tensor(I)
    return unitary

def marking_and_oracle(nqubits):
    """Returns a matrix representation of the and oracle"""
    unitary = sized_identity(nqubits)
    
    # Apply the x gate to the output register
    unitary.data[-1][-1] = 0
    unitary.data[-2][-2] = 0
    unitary.data[-1][-2] = 1
    unitary.data[-2][-1] = 1
    return unitary 

### Or

In [ ]:
def sized_x(nqubits):
    """Creates a not operator of given size"""
    unitary = X
    for i in range(nqubits-1):
        unitary = unitary.tensor(X)
    return unitary

def marking_or_oracle(nqubits):
    """Returns a matrix representation of the or oracle"""
    unitary = sized_x(nqubits).dot(marking_and_oracle(nqubits))
    unitary = unitary.dot(sized_x(nqubits-1).tensor(I))
    return unitary

While we could go on further and create _XOR_ and _AlternatingBits_ oracles but we don't need them for now so let's not :)

## SAT Oracles
Using our marking gate-oracles, we can create oracles for evaluating SAT clauses and finally complete functions.

### Single Clause Oracle
A Clause is a disjunction of variables (qubits) that are potentially negated.
The clause `x0 || !x1` may be represented by the input `[(0, true), (1, false)]`.

In [ ]:
import matplotlib.pyplot as plt

def marking_clause_oracle(qubits: [int], target: int, clause: [(int,bool)]):
    nqubits = len(qubits) + 1
    qc = QuantumCircuit(nqubits)
    # Flip input qubits
    for index, positive in clause:
        if not positive:
            qc.x(index)
            
    # Get Clause Qubits
    clause_qubits = [i for i, _ in clause]
    
    # Create an Oracle for n qubits
    nqubits_or = len(clause_qubits) + 1  # +1 for target qubit
    or_oracle = marking_or_oracle(nqubits_or)
    or_qc = QuantumCircuit(nqubits_or)
    or_qc.unitary(or_oracle, range(nqubits_or), "or_oracle")
    # compose list maps the qubits linearly (0,1,2,3...) on the given target list
    clause_qubits.append(target)
    return qc.compose(or_qc, clause_qubits)
#     return or_qc

In [ ]:
# Debug Clause Oracle
clause_circuit = marking_clause_oracle(range(5), 5, [(1, True),(3, False),(4, True)])
clause_circuit.draw(output="mpl")

### k-SAT Oracle
Using our single clause oracle, we can take their conjunction and create a marking oracle for the entire expression.

In [ ]:
def marking_ksat_oracle(qubits: [int], target: int, clauses: [[(int, bool)]]):
    auxiliary_qc = QuantumCircuit(len(clauses)) # Each clause output needs to be stored in aux
    qc = QuantumCircuit(len(qubits) + 1 + len(clauses))  # +1 for overall target
    
    # Compose individual clauses
    for idx, clause in enumerate(clauses):
        aux_idx = len(qubits) + idx
        one_clause_qc = marking_clause_oracle(qubits, len(qubits), clause)
        qbit_map = list(qubits)
        qbit_map.append(aux_idx)
        qc = qc.compose(one_clause_qc, qbit_map)
        
    # And the auxiliary registers
    nqubits_and = len(clauses) + 1
    and_oracle = marking_and_oracle(nqubits_and)
    and_qc = QuantumCircuit(nqubits_and)
    and_qc.unitary(and_oracle, range(nqubits_and), "and_oracle")
    
    # Compose the and oracle
    and_qc_map = [idx+len(qubits) for idx in range(len(clauses)+1)]
    # TODO Return auxiliary qubits to |0> State?
    return qc.compose(and_qc, and_qc_map)

def ksat_for_problem(problem):
    nqubits_aux = len(problem) # one for each clause
    nqubit_in = max([i for c in problem for i, _ in c]) + 1
    problem_qc = marking_ksat_oracle(range(nqubit_in), nqubit_in+nqubits_aux+1, problem)
    return problem_qc
    

**DONE** We can now formulate a SAT problem and create a marking oracle for it!

In [ ]:
# f(x) = (x1 + !x3 + x4) * (x0 + !x2)
problem = [[(1, True),(3, False),(4, True)],
          [(0, True),(2, False)]]
problem_circuit = ksat_for_problem(problem)
problem_circuit.draw(output="mpl")
